# 1. 데이터 준비

Word2Vec 사용시 scipy 버전 문제로 scipy 버전을 변경해야 한다.

> pip uninstall scipy

> pip install scipy==1.11.0

이후 재시작

## 1.1 다운로드

In [1]:
import urllib.request

urllib.request.urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt',
     filename='ratings.txt')

('ratings.txt', <http.client.HTTPMessage at 0x1e6909c3ef0>)

## 1.2 Dataframe 만들기

In [2]:
import pandas as pd 

train_data = pd.read_table('ratings.txt')
train_data[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


# 2. 데이터 전처리

## 2.1 결측치 확인

In [3]:
train_data.isnull().sum()

id          0
document    8
label       0
dtype: int64

## 2.2 결측치 제거

In [4]:
train_data = train_data.dropna(how='any')
train_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [5]:
train_data.shape

(199992, 3)

## 2.3 이상치 제거

In [6]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "")
train_data[:3]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1


## 2.4 불용어 제거

In [10]:
from konlpy.tag import Okt
from tqdm import tqdm

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

okt = Okt()
tokenized_data = []

for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True)
    stop_words_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    tokenized_data.append(stop_words_removed_sentence)

100%|██████████| 199992/199992 [08:13<00:00, 405.37it/s]


In [8]:
tokenized_data[:5]

[['어리다', '때', '보고', '지금', '다시', '보다', '재밌다', 'ㅋㅋ'],
 ['디자인',
  '을',
  '배우다',
  '학생',
  ',',
  '외국',
  '디자이너',
  '그',
  '일군',
  '전통',
  '을',
  '통해',
  '발전',
  '문화',
  '산업',
  '부럽다',
  '.',
  '사실',
  '우리나라',
  '에서도',
  '그',
  '어렵다',
  '시절',
  '끝',
  '까지',
  '열정',
  '을',
  '지키다',
  '노라노',
  '같다',
  '전통',
  '있다',
  '저',
  '같다',
  '사람',
  '꿈',
  '을',
  '꾸다',
  '이루다',
  '나가다',
  '수',
  '있다',
  '것',
  '감사하다',
  '.'],
 ['폴리스스토리', '시리즈', '1', '부터', '뉴', '까지', '버리다', '하나', '없다', '..', '최고', '.'],
 ['오다',
  '..',
  '연기',
  '진짜',
  '개',
  '쩔다',
  '..',
  '지루하다',
  '생각',
  '몰입',
  '보다',
  '..',
  '그렇다',
  '이렇다',
  '진짜',
  '영화',
  '지'],
 ['안개', '자욱하다', '밤하늘', '뜨다', '있다', '초승달', '같다', '영화', '.']]

# 3. 모델 만들기

## 3.1 훈련시키기

## Word2Vec 패키지 설치
```
pip install gensim
```

In [11]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=tokenized_data, window=5, min_count=5, workers=4, sg=0)

## 3.2 크기 확인

In [12]:
model.wv.vectors.shape

(17615, 100)

## 3.3 유사도 확인

In [13]:
print(model.wv.most_similar("최민식"))

[('윤제문', 0.824195146560669), ('한석규', 0.8139277696609497), ('김명민', 0.812897264957428), ('송강호', 0.8122984170913696), ('서영희', 0.8088347911834717), ('최민수', 0.8070887923240662), ('워싱턴', 0.7943331599235535), ('조인성', 0.7938597202301025), ('메릴', 0.7934570908546448), ('이정재', 0.7931817173957825)]


In [14]:
print(model.wv.most_similar("히어로"))

[('슬래셔', 0.8522522449493408), ('무협', 0.8447504639625549), ('호러', 0.8427553772926331), ('블록버스터', 0.8190979361534119), ('물', 0.806109607219696), ('무비', 0.801958441734314), ('SF', 0.7974154353141785), ('정통', 0.7935965061187744), ('물의', 0.788445770740509), ('sf', 0.7800335884094238)]


In [16]:
print(model.wv.most_similar("배"))

[('배다', 0.6515790820121765), ('번하다', 0.6085609197616577), ('배속', 0.6067914366722107), ('번은', 0.5926095843315125), ('글자', 0.5867164134979248), ('마리', 0.580321192741394), ('번봄', 0.572825014591217), ('부작', 0.5688346028327942), ('라운드', 0.5667506456375122), ('편모', 0.56169193983078)]


In [17]:
print(model.wv.most_similar("이정재"))

[('안성기', 0.8615968823432922), ('정재영', 0.8599830865859985), ('최민수', 0.8556740880012512), ('공효진', 0.848192036151886), ('송강호', 0.8422252535820007), ('조인성', 0.8398154377937317), ('박중훈', 0.83797287940979), ('심은하', 0.8355767726898193), ('김혜수', 0.8305727243423462), ('신하균', 0.8292660117149353)]


In [18]:
print(model.wv.most_similar("이순신"))

[('김두영', 0.7986792922019958), ('크리스토퍼', 0.7960168719291687), ('뤽', 0.7946228981018066), ('한경직', 0.7925119400024414), ('신화', 0.7905298471450806), ('선교사', 0.7898854613304138), ('래퍼', 0.7848513126373291), ('이덕화', 0.7828461527824402), ('핀처', 0.7816781401634216), ('윌리엄', 0.7806227207183838)]


In [19]:
print(model.wv.most_similar("코믹"))

[('코미디', 0.7727064490318298), ('멜로', 0.7341834902763367), ('스릴러', 0.7228541970252991), ('개그', 0.7090107202529907), ('로맨틱', 0.702569305896759), ('호러', 0.6900100111961365), ('코메디', 0.6787692308425903), ('서스펜스', 0.6700559258460999), ('유머', 0.6672298312187195), ('로맨스', 0.6648151874542236)]


# 참조
- https://wikidocs.net/50739
- https://skillmemory.tistory.com/17